In [22]:
# Data handling
import numpy as np
import pandas as pd
import re

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
%matplotlib inline

#lib
from lib.clean_data_functions import clean_ext_version, clean_ext_publisher, clean_repo_publisher

#
from ast import literal_eval
from collections import Counter

# os
import os

# time
import time

import warnings
warnings.filterwarnings("ignore")    # (Optional)

print("Project has been created with Pandas: " ,pd. __version__," And with Numpy: ",np. __version__)

Project has been created with Pandas:  2.3.1  And with Numpy:  2.3.2


In [23]:
import yaml

try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

### 1. Loading

In [24]:
df = pd.read_csv(config["data"]["raw"]["file_ext_repo"])
# df = df.sort_values(by = ["ext_install_count", "ext_rating"], ascending= False)
df.head()

,ext_name,ext_publisher,ext_version,ext_categories,ext_tags,ext_install_count,ext_rating,ext_last_updated,repo_publisher,verified,...,high,medium,low,repo_owner,repo_name,repo_stars,repo_forks,language,topics,error
0,Python,ms-python,2025.11.2025072901,Programming Languages;Debuggers;Data Science;M...,__ext_j2;__ext_jinja2;__web_extension;debugger...,177844528,4.192869,2025-07-29T10:50:24.31Z,Microsoft,True,...,0,0,0,Microsoft,vscode-python,4461,1247,"[""TypeScript"",""Python"",""JavaScript"",""Jupyter N...",[],NaN
1,Pylance,ms-python,2025.7.100,Programming Languages,__web_extension;json;python,146473811,3.007722,2025-07-30T23:10:58.657Z,Microsoft,True,...,0,0,0,microsoft,pylance-release,1882,856,"[""Python"",""Jupyter Notebook""]","[""python"",""language-server"",""code-analysis"",""l...",NaN
2,Jupyter,ms-toolsai,2025.7.2025073101,Extension Packs;Data Science;Machine Learning;...,__ext_jl;__web_extension;debuggers;interactive...,93931981,2.700599,2025-07-31T09:54:06Z,Microsoft,True,...,0,0,0,Microsoft,vscode-jupyter,1401,337,"[""TypeScript"",""Python"",""Jupyter Notebook"",""Jav...","[""machine-learning"",""jupyter"",""vscode"",""datasc...",NaN
3,C/C++,ms-vscode,1.26.3,Programming Languages;Snippets;Linters;Debugge...,C;C++;cpp;cuda-cpp;debuggers;IntelliSense;json...,85413454,3.426871,2025-06-25T19:02:16.243Z,Microsoft,True,...,0,0,11,Microsoft,vscode-cpptools,5873,1633,"[""TypeScript"",""C++"",""HTML"",""JavaScript""]","[""microsoft"",""typescript"",""vscode-extension""]",NaN
4,Jupyter Keymap,ms-toolsai,1.1.2,Notebooks,__web_extension;keybindings;notebook-keymap,74084334,4.000000,2023-06-05T17:53:31.993Z,Microsoft,True,...,0,9,8,Microsoft,vscode-jupyter-keymap,32,14,[],[],NaN


In [25]:
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns')

The dataset has 24396 rows and 23 columns


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24396 entries, 0 to 24395
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ext_name               24396 non-null  object 
 1   ext_publisher          24396 non-null  object 
 2   ext_version            24396 non-null  object 
 3   ext_categories         24396 non-null  object 
 4   ext_tags               20469 non-null  object 
 5   ext_install_count      24396 non-null  int64  
 6   ext_rating             24396 non-null  float64
 7   ext_last_updated       24396 non-null  object 
 8   repo_publisher         24396 non-null  object 
 9   verified               24396 non-null  bool   
 10  repository             24396 non-null  object 
 11  total_vulnerabilities  24396 non-null  int64  
 12  critical               24396 non-null  int64  
 13  high                   24396 non-null  int64  
 14  medium                 24396 non-null  int64  
 15  lo

#### Metadata:
- **_verified_:**                 check, whether extension's security is breached         (boolean)
- **_ext_name_:**                 name of extension                                       (obj)
- **_ext_publisher_:**            name of extension'S publisher                           (obj)
- **_ext_version_:**              current version of extension                            (obj)
- **_ext_categories_:**           categories of extension                                 (obj)   (multi values)
- **_ext_tags_:**                 keywords related to extension                           (obj)   (multi values)
- **_ext\_install\_count_:**      total number of installations of extension              (int64)
- **_ext\_rating_:**              rating of extension (avg of stars rating)               (float64)
- **_ext\_last\_updated_:**       timestamp of last update                                (obj)
- **_repo_publisher_:**           publisher of extension                                  (obj)
- **_repository_:**               url of repository                                       (obj)
- **_total_vulnerabilities_:**    number of detected vulnerabilities                      (int64)
- **_critical_:**                 number of critical(severity) vulnerabilities            (int64)
- **_high_:**                     number of high(severity) vulnerabilities                (int64)
- **_medium_:**                   number of medium(severity) vulnerabilities              (int64)
- **_low_:**                      number of low(severity) vulnerabilities                 (int64)
- **_repo\_owner_:**              owner of repository (via column repository)             (obj)
- **_repo\_name_:**               name of repository (via column repository)              (obj)
- **_repo\_stars_:**              number of stars of repository (via column repository)   (int64)   
- **_repo\_forks_:**              number of forks of repository (via column repository)   (int64)   
- **_language_:**                 program languages used (via column repository)          (obj)   (multi values)
- **_topics_:**                   keywords related to repository (via column repository)  (obj)   (multi values)
- **_error_:**                    log of fetching repository                              (obj)


### 2. Cleaning

In [27]:
# Check missing values
df.isna().sum()

ext_name                     0
ext_publisher                0
ext_version                  0
ext_categories               0
ext_tags                  3927
ext_install_count            0
ext_rating                   0
ext_last_updated             0
repo_publisher               0
verified                     0
repository                   0
total_vulnerabilities        0
critical                     0
high                         0
medium                       0
low                          0
repo_owner                   0
repo_name                    0
repo_stars                   0
repo_forks                   0
language                     0
topics                       0
error                    21601
dtype: int64

In [28]:
df.columns[df.isnull().mean() > 0.8]

Index(['error'], dtype='object')

Remove record with values (not null) in "error" column due to unavailability of repository

In [29]:
df = df[df["error"].isna()]
print(f"After removing extensions with unavailable repository, the dataset has {df.shape[0]} rows and {df.shape[1]} columns")

After removing extensions with unavailable repository, the dataset has 21601 rows and 23 columns


#### Clean columns

Rename and remove columns for noises and avoiding of overfit ["ext\_tags", "repo\_publisher", "error"]

In [30]:
# Use  "repo\_owner" over "repo\_publisher" due to correctness from fetching infos directly)
df = df.drop(columns=["ext_name","ext_publisher","ext_tags","ext_last_updated", "repo_publisher", "error"]).rename(columns={"total_vulnerabilities": "total_vulners", "critical": "critical__vulners", "high": "high__vulners", "medium": "medium__vulners", "low": "low__vulners", "language": "repo_languages", "topics": "repo_topics"})

In [31]:
df

,ext_version,ext_categories,ext_install_count,ext_rating,verified,repository,total_vulners,critical__vulners,high__vulners,medium__vulners,low__vulners,repo_owner,repo_name,repo_stars,repo_forks,repo_languages,repo_topics
0,2025.11.2025072901,Programming Languages;Debuggers;Data Science;M...,177844528,4.192869,True,https://github.com/Microsoft/vscode-python,0,0,0,0,0,Microsoft,vscode-python,4461,1247,"[""TypeScript"",""Python"",""JavaScript"",""Jupyter N...",[]
1,2025.7.100,Programming Languages,146473811,3.007722,True,https://github.com/microsoft/pylance-release,0,0,0,0,0,microsoft,pylance-release,1882,856,"[""Python"",""Jupyter Notebook""]","[""python"",""language-server"",""code-analysis"",""l..."
2,2025.7.2025073101,Extension Packs;Data Science;Machine Learning;...,93931981,2.700599,True,https://github.com/Microsoft/vscode-jupyter,0,0,0,0,0,Microsoft,vscode-jupyter,1401,337,"[""TypeScript"",""Python"",""Jupyter Notebook"",""Jav...","[""machine-learning"",""jupyter"",""vscode"",""datasc..."
3,1.26.3,Programming Languages;Snippets;Linters;Debugge...,85413454,3.426871,True,https://github.com/Microsoft/vscode-cpptools,11,0,0,0,11,Microsoft,vscode-cpptools,5873,1633,"[""TypeScript"",""C++"",""HTML"",""JavaScript""]","[""microsoft"",""typescript"",""vscode-extension""]"
4,1.1.2,Notebooks,74084334,4.000000,True,https://github.com/Microsoft/vscode-jupyter-ke...,17,0,0,9,8,Microsoft,vscode-jupyter-keymap,32,14,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24389,0.0.1,Themes,1,0.000000,False,https://github.com/finnlinn/uwustyl,0,0,0,0,0,finnlinn,uwustyl,0,0,[],[]
24390,1.0.1,Other,2,0.000000,False,https://github.com/akainth015/marscode,0,0,0,0,0,akainth015,marscode,1,1,"[""TypeScript""]",[]
24392,1.0.2,Other,1,0.000000,True,https://github.com/Onboardbase/onboardbase-vscode,1,0,1,0,0,Onboardbase,onboardbase-vscode,3,0,"[""TypeScript"",""JavaScript""]",[]
24394,0.4.2,Debuggers,0,0.000000,True,https://github.com/ms-iot/vscode-ros,4,0,0,0,4,ms-iot,vscode-ros,427,103,"[""TypeScript"",""Python"",""HTML"",""JavaScript"",""CSS""]","[""ros""]"


In [32]:
# df.info()

#### Clean data inconsistencies

In [33]:
#ext_version

df["ext_version"] = df["ext_version"].apply(clean_ext_version)
df['ext_version'] = pd.to_numeric(df['ext_version'], errors="coerce")

In [34]:
#ext_rating
df["ext_rating"] = df["ext_rating"].apply(lambda x: round(x,2) if pd.notna(x) else pd.NA )

In [35]:
# df[(df["ext_version"] > 10) & (df["ext_rating"] == 0) & (df["ext_install_count"] < 100) & (df["repo_stars"] < 10)]
# df[(df["ext_rating"] == 0) & (df["ext_install_count"] < 10) & (df["repo_stars"] < 10)]
df

,ext_version,ext_categories,ext_install_count,ext_rating,verified,repository,total_vulners,critical__vulners,high__vulners,medium__vulners,low__vulners,repo_owner,repo_name,repo_stars,repo_forks,repo_languages,repo_topics
0,2025,Programming Languages;Debuggers;Data Science;M...,177844528,4.19,True,https://github.com/Microsoft/vscode-python,0,0,0,0,0,Microsoft,vscode-python,4461,1247,"[""TypeScript"",""Python"",""JavaScript"",""Jupyter N...",[]
1,2025,Programming Languages,146473811,3.01,True,https://github.com/microsoft/pylance-release,0,0,0,0,0,microsoft,pylance-release,1882,856,"[""Python"",""Jupyter Notebook""]","[""python"",""language-server"",""code-analysis"",""l..."
2,2025,Extension Packs;Data Science;Machine Learning;...,93931981,2.70,True,https://github.com/Microsoft/vscode-jupyter,0,0,0,0,0,Microsoft,vscode-jupyter,1401,337,"[""TypeScript"",""Python"",""Jupyter Notebook"",""Jav...","[""machine-learning"",""jupyter"",""vscode"",""datasc..."
3,1,Programming Languages;Snippets;Linters;Debugge...,85413454,3.43,True,https://github.com/Microsoft/vscode-cpptools,11,0,0,0,11,Microsoft,vscode-cpptools,5873,1633,"[""TypeScript"",""C++"",""HTML"",""JavaScript""]","[""microsoft"",""typescript"",""vscode-extension""]"
4,1,Notebooks,74084334,4.00,True,https://github.com/Microsoft/vscode-jupyter-ke...,17,0,0,9,8,Microsoft,vscode-jupyter-keymap,32,14,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24389,0,Themes,1,0.00,False,https://github.com/finnlinn/uwustyl,0,0,0,0,0,finnlinn,uwustyl,0,0,[],[]
24390,1,Other,2,0.00,False,https://github.com/akainth015/marscode,0,0,0,0,0,akainth015,marscode,1,1,"[""TypeScript""]",[]
24392,1,Other,1,0.00,True,https://github.com/Onboardbase/onboardbase-vscode,1,0,1,0,0,Onboardbase,onboardbase-vscode,3,0,"[""TypeScript"",""JavaScript""]",[]
24394,0,Debuggers,0,0.00,True,https://github.com/ms-iot/vscode-ros,4,0,0,0,4,ms-iot,vscode-ros,427,103,"[""TypeScript"",""Python"",""HTML"",""JavaScript"",""CSS""]","[""ros""]"


In [36]:
#repo_languages
# eval the obj[] and transform to str
df["repo_languages"] = df["repo_languages"].apply(lambda x: literal_eval(x) if pd.notna(x) else [])
df["repo_languages"] = df["repo_languages"].apply(lambda x: ';'.join(map(str, x)) if x else x)
df["repo_languages"] = df["repo_languages"].str.lower()
# prog_languages = languages_set

In [37]:
#repo_topics
df["repo_topics"] = df["repo_topics"].apply(lambda x: literal_eval(x) if pd.notna(x) else [])
df["repo_topics"] = df["repo_topics"].apply(lambda x: ';'.join(map(str, x)) if x else pd.NA)
df["repo_topics"] = df["repo_topics"].str.lower()


In [38]:
# df["repo_languages"] = df["repo_languages"].where(~(pd.isna(df["repo_languages"]) & ~pd.isna(df["repo_topics"]) & df["repo_topics"].isin(list(languages_set))), df["repo_topics"])

In [39]:
df

,ext_version,ext_categories,ext_install_count,ext_rating,verified,repository,total_vulners,critical__vulners,high__vulners,medium__vulners,low__vulners,repo_owner,repo_name,repo_stars,repo_forks,repo_languages,repo_topics
0,2025,Programming Languages;Debuggers;Data Science;M...,177844528,4.19,True,https://github.com/Microsoft/vscode-python,0,0,0,0,0,Microsoft,vscode-python,4461,1247,typescript;python;javascript;jupyter notebook;...,<NA>
1,2025,Programming Languages,146473811,3.01,True,https://github.com/microsoft/pylance-release,0,0,0,0,0,microsoft,pylance-release,1882,856,python;jupyter notebook,python;language-server;code-analysis;language-...
2,2025,Extension Packs;Data Science;Machine Learning;...,93931981,2.70,True,https://github.com/Microsoft/vscode-jupyter,0,0,0,0,0,Microsoft,vscode-jupyter,1401,337,typescript;python;jupyter notebook;javascript;...,machine-learning;jupyter;vscode;datascience;vs...
3,1,Programming Languages;Snippets;Linters;Debugge...,85413454,3.43,True,https://github.com/Microsoft/vscode-cpptools,11,0,0,0,11,Microsoft,vscode-cpptools,5873,1633,typescript;c++;html;javascript,microsoft;typescript;vscode-extension
4,1,Notebooks,74084334,4.00,True,https://github.com/Microsoft/vscode-jupyter-ke...,17,0,0,9,8,Microsoft,vscode-jupyter-keymap,32,14,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24389,0,Themes,1,0.00,False,https://github.com/finnlinn/uwustyl,0,0,0,0,0,finnlinn,uwustyl,0,0,NaN,<NA>
24390,1,Other,2,0.00,False,https://github.com/akainth015/marscode,0,0,0,0,0,akainth015,marscode,1,1,typescript,<NA>
24392,1,Other,1,0.00,True,https://github.com/Onboardbase/onboardbase-vscode,1,0,1,0,0,Onboardbase,onboardbase-vscode,3,0,typescript;javascript,<NA>
24394,0,Debuggers,0,0.00,True,https://github.com/ms-iot/vscode-ros,4,0,0,0,4,ms-iot,vscode-ros,427,103,typescript;python;html;javascript;css,ros


In [40]:
repo_languages = ""
for lang in df["repo_languages"]:
    if not pd.isna(lang):
        repo_languages += lang + ";"
    
repo_languages_list = repo_languages.split(';')
repo_languages_set  = set(repo_languages_list)

In [41]:
def filter_string(s, valid_set):
    if pd.isna(s):
        return np.nan
    return ';'.join([x for x in s.split(';') if x in valid_set])

# Fill NA in language where topics is not NA with parts of topics in languages
mask = pd.isna(df['repo_languages']) & ~pd.isna(df['repo_topics'])
df.loc[mask, 'repo_languages'] = df.loc[mask, 'repo_topics'].apply(lambda x: filter_string(x, repo_languages_set))

In [42]:
df["repo_languages"].isna().sum()/df["repo_languages"].shape[0]
# sorted(languages_set)

np.float64(0.24559048192213323)

NA in column "repo_languages" is 24,6%. We will try to fillna with mode() of languages\_list or unknown ??

#### Handle missing/na values

In [43]:
df["repo_languages"] = df["repo_languages"].fillna("unknown")

#### Handle duplicating


In [44]:
# df.duplicated().sum()
df = df[~df.duplicated(subset=["repo_owner","repo_stars", "repo_forks"])]

#### Transform

In [45]:
# df[df["repo_languages"].isna()]
df = df.drop(columns="repo_topics")
# df[(df["repo_languages"].isna()) & df["verified"] == True]
df

,ext_version,ext_categories,ext_install_count,ext_rating,verified,repository,total_vulners,critical__vulners,high__vulners,medium__vulners,low__vulners,repo_owner,repo_name,repo_stars,repo_forks,repo_languages
0,2025,Programming Languages;Debuggers;Data Science;M...,177844528,4.19,True,https://github.com/Microsoft/vscode-python,0,0,0,0,0,Microsoft,vscode-python,4461,1247,typescript;python;javascript;jupyter notebook;...
1,2025,Programming Languages,146473811,3.01,True,https://github.com/microsoft/pylance-release,0,0,0,0,0,microsoft,pylance-release,1882,856,python;jupyter notebook
2,2025,Extension Packs;Data Science;Machine Learning;...,93931981,2.70,True,https://github.com/Microsoft/vscode-jupyter,0,0,0,0,0,Microsoft,vscode-jupyter,1401,337,typescript;python;jupyter notebook;javascript;...
3,1,Programming Languages;Snippets;Linters;Debugge...,85413454,3.43,True,https://github.com/Microsoft/vscode-cpptools,11,0,0,0,11,Microsoft,vscode-cpptools,5873,1633,typescript;c++;html;javascript
4,1,Notebooks,74084334,4.00,True,https://github.com/Microsoft/vscode-jupyter-ke...,17,0,0,9,8,Microsoft,vscode-jupyter-keymap,32,14,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385,0,Other,4,0.00,False,https://github.com/longnlp/OneBlueCodeExtension,0,0,0,0,0,longnlp,OneBlueCodeExtension,0,0,typescript
24387,1,Other,3,0.00,False,https://github.com/rainte/vscode,0,0,0,0,0,rainte,vscode,0,0,typescript;javascript
24388,0,Other,3,0.00,False,https://github.com/bra1nDump/bread,0,0,0,0,0,bra1nDump,ai-task-vscode,42,6,typescript;javascript;shell
24390,1,Other,2,0.00,False,https://github.com/akainth015/marscode,0,0,0,0,0,akainth015,marscode,1,1,typescript


In [46]:
df.to_csv(config["data"]["clean"]["file_data_cleaned"],index=False )

In [47]:
# languages = ""
# for lang in df["repo_languages"]:
#     if not pd.isna(lang):
#         languages += lang + ";"
    
# languages_list = languages.split(';')
# languages_set  = set(languages_list)

In [48]:

# categories = ""
# for cate in df["ext_categories"]:
#     categories += cate + ";"

# categories_list = categories.split(';')
# categories_set  = set(categories_list)

# categories_set